In [1]:
import pandas as pd; import numpy as np; import os;import random;import time;

In [2]:
import random; import gzip;

# Explorar geo datos de llamados (Voice)
en un principio si los datos no son geolocalizados por antena no puedo saber quien llamo a la zona endemica. El objetivo es ver...

* como se compone la base de datos
* que sirve para el analisis de la mobilidad y del "Gran Chaco"
* haria falta tambien tener el .cpp que asigna home a cada user


In [3]:
np.random.seed(2015)

In [4]:
!ls 

Antennas department and epidemic.ipynb
Antennas plot??.ipynb
SimpleFormat
SimpleFormat data structure explroation.ipynb
Testeo de geo voice dataset.ipynb
antennas_mexico.csv
datosgeo
mapas
mapas-mex.tar.gz
output_memory_summary.txt
output_time.txt
scripts-antennas-preprocessing
simple_format.sh
simple_format_Prepago_01-020115.txt.gz
simple_format_Prepago_03-040115.txt.gz
simple_format_Prepago_17-180115.txt.gz
surrogated_antennas.txt.gz
surrogated_nodes.txt.gz


In [8]:
tabla = pd.read_csv("./datosgeo/Prepago/Geo_Voz_Prepago_03-040115.txt.gz",sep="\t")

In [9]:
tabla.head()

,CONTRATO,TELEFONO,ORIGEN,DESTINO,DIRECCION,DURACION_SEGUNDOS,UBICACIONCIUDAD,LATITUD,LONGITUD,OPERADOR,FECHA_LLAMADA
0,NaN,00000837B868F926F67BD440A0845C1D,00000837B868F926F67BD440A0845C1D,47D8EF4D60C18D481029AB43899DE0CE,ENTRANTE,14,Del Tlahuac,19.289000,-99.006000,UNEFON,04/01/2015 08:55:03 p.m.
1,NaN,00000837B868F926F67BD440A0845C1D,00000837B868F926F67BD440A0845C1D,47D8EF4D60C18D481029AB43899DE0CE,ENTRANTE,37,Del Tlahuac,19.289000,-99.006000,UNEFON,04/01/2015 07:17:21 p.m.
2,NaN,00000837B868F926F67BD440A0845C1D,00000837B868F926F67BD440A0845C1D,6A02905E6E30925EED11DC2F1EF6A3C2,ENTRANTE,34,Del Tlahuac,19.289000,-99.006000,UNEFON,04/01/2015 08:21:10 a.m.
3,NaN,00000837B868F926F67BD440A0845C1D,00000837B868F926F67BD440A0845C1D,7973538E2B7E5B6450EE332D2193A8B4,ENTRANTE,28,Del Tlahuac,19.289000,-99.006000,UNEFON,04/01/2015 06:12:15 p.m.
4,NaN,0000129F4191FFD0963774D2307C7510,0000129F4191FFD0963774D2307C7510,B55022400C295828668E644CFDA58425,ENTRANTE,75,Tehuacan,18.453695,-97.401057,UNEFON,04/01/2015 08:43:27 p.m.


In [10]:
#para ver valores unicos lo transformamos a un conjunto
set(tabla.OPERADOR.values)

{nan,
 'ALESTRA           ',
 'AVANTEL           ',
 'AXTEL             ',
 'BESTEL            ',
 'BESTPHONE         ',
 'CABLECOM          ',
 'CABLEMAS          ',
 'IP MATRIX         ',
 'IUSACELL          ',
 'IUSATEL           ',
 'MARCATELCOM       ',
 'MAXCOM            ',
 'MEGA CABLE        ',
 'MEGACABLE (MCM)   ',
 'METRORED          ',
 'MOVISTAR          ',
 'NO IDENTIFICADO   ',
 'OPCOM             ',
 'OPERBES           ',
 'PROTEL I-NEXT     ',
 'PROTELOP          ',
 'TELCEL            ',
 'TELECABLE         ',
 'TELEFONICA        ',
 'TELMEX            ',
 'TELNOR            ',
 'TOTAL PLAY        ',
 'UNEFON            '}

## Obs.
* estan cambiadas las columnas ORIGEN y DESTINO 
* CONTRATO es todo NaNs
* OPERADORA y DURACION_SEGUNDOS no aportan info, 

In [14]:
tabla.columns

Index([u'TELEFONO', u'DESTINO', u'ORIGEN', u'DIRECCION', u'UBICACIONCIUDAD',
       u'LATITUD', u'LONGITUD', u'OPERADOR', u'FECHA_LLAMADA'],
      dtype='object')

In [14]:
tabla.shape

(7860676, 7)

In [10]:
tabla.rename(columns={'ORIGEN': 'DESTINO', 'DESTINO': 'ORIGEN'},inplace=True);
#aunque suene extranyo tambien voy a tirar la columna ubicacion/ciudad porque tiene muchos NaNs y 
#porque solo miro la info a nivel 'departamental'
tabla.drop(['CONTRATO','OPERADOR','DURACION_SEGUNDOS','UBICACIONCIUDAD'],axis=1,inplace=True)


In [13]:
#chequeo si hay info faltante
tabla[pd.isnull(tabla).any(axis=1)]
#resulta que hay como 600k de casi 8M

,TELEFONO,DESTINO,ORIGEN,DIRECCION,LATITUD,LONGITUD,FECHA_LLAMADA
26,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,C2430855E8747E6F08EB709DE21FED58,ENTRANTE,NaN,NaN,03/01/2015 08:09:34 a.m.
27,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,C2430855E8747E6F08EB709DE21FED58,ENTRANTE,NaN,NaN,03/01/2015 06:06:42 a.m.
28,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,F687EB84F1786572AF0CC7C4BA1047D8,ENTRANTE,NaN,NaN,03/01/2015 06:03:35 a.m.
353,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,57A17524BF266EEAA7AA425BDB58EC23,ENTRANTE,NaN,NaN,03/01/2015 06:42:18 p.m.
354,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 05:19:30 p.m.
355,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:15:42 p.m.
356,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:16:47 p.m.
357,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:15:07 p.m.
358,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,B1475C3F83AA1CC6E5969CED45F6EBE9,ENTRANTE,NaN,NaN,03/01/2015 07:04:33 p.m.
362,0001B96F3892319D191BCEEAE0AD48DD,0001B96F3892319D191BCEEAE0AD48DD,70C00DE41AB64A5720E59BDB6BE0BC05,ENTRANTE,NaN,NaN,04/01/2015 10:04:14 a.m.


In [16]:
tabla.dropna(inplace=True)

In [17]:
tabla2 = pd.read_csv("datosgeo/Prepago/Geo_Voz_Prepago_01-020115.txt.gz",sep="\t")
tabla2.drop(['CONTRATO','OPERADOR','DURACION_SEGUNDOS','UBICACIONCIUDAD'],axis=1,inplace=True)
tabla2.rename(columns={'ORIGEN': 'DESTINO', 'DESTINO': 'ORIGEN'},inplace=True);
tabla2.dropna(inplace=True)

In [18]:
sample=tabla.ix[random.sample(tabla.index, 3*(10**5))]
sample2=tabla2.ix[random.sample(tabla2.index, 3*(10**5))]


In [ ]:
del tabla; del tabla2

In [19]:
sample.index = range(1,len(sample)+1)
sample2.index = range(1,len(sample2)+1)

In [20]:
test2=sample.ix[100].FECHA_LLAMADA
test2[0:-3]+'m'

'04/01/2015 12:21:05 pm'

In [20]:
import time;

#### paso el date a formato epoch

In [21]:
sample['FECHA_LLAMADA'] = sample['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));
sample2['FECHA_LLAMADA'] = sample2['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));

In [22]:
sample.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);
sample2.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);

In [13]:
#hago un sort por el FECHA_LLAMADA para ver cuando arranca la base de datos 
tabla.index = range(1,len(tabla)+1)

In [16]:
tabla['FECHA_LLAMADA'] = tabla['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));
tabla.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);

In [17]:
tabla.sort_values(by='FECHA_LLAMADA_EPOCH',ascending=True,inplace=True)

In [18]:
tabla.head()

,TELEFONO,DESTINO,ORIGEN,DIRECCION,UBICACIONCIUDAD,LATITUD,LONGITUD,FECHA_LLAMADA_EPOCH
5015223,DC1F8BC68A6B6D22A7CBD4C9787BD9DB,DC1F8BC68A6B6D22A7CBD4C9787BD9DB,F1BB45B86CC4540E6194D6A28F177825,SALIENTE,Tlalnepantla,19.517500,-99.102500,1420070400
6984754,CBF4C39BE4C0EAC246FCD5D206C5D2E4,CBF4C39BE4C0EAC246FCD5D206C5D2E4,9877CC267054BD7E5871D9D5627FD507,ENTRANTE,NaN,18.966750,-99.247667,1420070400
10363183,E3595DB849B1CD0FC19187317ECA639F,E3595DB849B1CD0FC19187317ECA639F,139AD14F044D03AEE227024E84A615C6,ENTRANTE,Apan,19.710194,-98.448611,1420070400
9821410,76D12717E57C739DB1C2AE5ADA26BC92,76D12717E57C739DB1C2AE5ADA26BC92,30A61DC885EDDEA18B8AB707E5A84321,ENTRANTE,Guadalajara,20.670161,-103.382536,1420070400
6754731,501EF14B895B19BF0F6034C310EF85E0,501EF14B895B19BF0F6034C310EF85E0,2843788532C31620AF795210D4E8D891,ENTRANTE,Xonacatlan,19.405165,-99.521802,1420070400


# *Obs* 
no veo nada porque aca arranca desde EPOCH... PERO mirando la documentacion del trac resulta que todo esta seteado para arrancar el 01/01/2012 a las 0 horas